In [1]:
import numpy as np
from scipy import io
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from ST_GCN import Model
from sklearn.model_selection import ShuffleSplit
from torch.utils.data import Dataset,DataLoader
from EarlyStopping import EarlyStopping
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
def get_model(adj_hat_path_):
    temp_k=7
    net = Model(1, 2, True, temp_k, adj_hat_path_)
    net=net.to(device)
    optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.0001)
    return net, optimizer

def train_model(net,optimizer,criterion,train_dataloader):
    training_loss,train_acc = 0.0,0.0
    net.train()
    for data, label in train_dataloader:
        data=data.to(device)
        label=label.to(device)
        optimizer.zero_grad()
        outputs = net(data)
        loss = criterion(outputs, label.long())
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
        outputs = outputs.argmax(dim=1).data.cpu().numpy()
        train_acc+=np.mean(outputs == label.cpu().numpy())
    return train_acc/len(train_dataloader),training_loss/len(train_dataloader)

def test_model(net,criterion,test_dataloader):
    test_loss,test_acc = 0.0,0.0
    net.eval()
    for data, label in test_dataloader:
        data=data.to(device)
        label=label.to(device)
        outputs = net(data)
        test_loss +=criterion(outputs, label.long()).item()
        outputs = outputs.argmax(dim=1).data.cpu().numpy()
        test_acc += np.mean(outputs == label.cpu().numpy())
    return test_acc/len(test_dataloader),test_loss/len(test_dataloader)

#change dataloader by yourself
class MyDataset(Dataset):
    def __init__(self, data_len):
        self.data=torch.rand((data_len,1,900,136))#B,C,T,V
        self.label=torch.ones(data_len)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        data=self.data[idx]
        label=self.label[idx]
        return data,label
#create adj
adj = np.corrcoef(np.random.rand(136,170*900))
np.save('adj.npy',adj,allow_pickle=True)

In [3]:
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)

train_dataset=MyDataset(170)
train_dataloader=DataLoader(train_dataset,batch_size=16,shuffle=True)
test_dataset=MyDataset(10)
test_dataloader=DataLoader(test_dataset,batch_size=1)

tra_acc,tra_loss,tes_acc,tes_loss=[],[],[],[]
patience = 20
early_stopping = EarlyStopping(patience, verbose=True)
criterion = nn.CrossEntropyLoss() # nn.BCELoss()
net,optimizer = get_model('adj.npy')
if_best = False
edge_imp_all = []
for epoch in range(1, 501):
    acc,loss = train_model(net,optimizer,criterion,train_dataloader)
    tra_acc.append(acc)
    tra_loss.append(tra_loss)
    acc,loss = test_model(net,criterion,test_dataloader)
    tes_acc.append(acc)
    tes_loss.append(loss)
    if epoch>200:
        for importance in net.edge_importance:
            edge_importances = (torch.abs(importance)+torch.transpose(torch.abs(importance),0,1))*0.5
            edge_imp = torch.squeeze(edge_importances.data).cpu().numpy()
            edge_imp_all.append(edge_imp)
        if_best = early_stopping(loss, net)#return True if early stop
    if if_best:
        best_test_acc = acc
        torch.save(net.state_dict(),'saved.pt')
    if early_stopping.early_stop:
        edge_imp = np.array(edge_imp_all).mean(axis=0)
        print('stop epoch:',epoch,"Early stopping, best test acc:",best_test_acc)
        # 结束模型训练
        break

stop epoch: 366 Early stopping, best test acc: 1.0
1.0
